In [1]:
import os
import cv2
import numpy as np
import argparse
import torch as th
import torch.nn.functional as F
import time
import conf_mgt
from utils import yamlread
from guided_diffusion import dist_util

try:
    import ctypes
    libgcc_s = ctypes.CDLL('libgcc_s.so.1')
except:
    pass

from guided_diffusion.script_util import (
    NUM_CLASSES,
    model_and_diffusion_defaults,
    classifier_defaults,
    create_model_and_diffusion,
    create_classifier,
    select_args,
)

def toU8(sample):
    if sample is None:
        return sample

    sample = ((sample + 1) * 127.5).clamp(0, 255).to(th.uint8)
    sample = sample.permute(0, 2, 3, 1)
    sample = sample.contiguous()
    sample = sample.detach().cpu().numpy()
    return sample

def main(conf_path):

    conf = conf_mgt.conf_base.Default_Conf()
    conf.update(yamlread(conf_path))

    print("Start", conf['name'])

    device = dist_util.dev(conf.get('device'))

    model, diffusion = create_model_and_diffusion(
        **select_args(conf, model_and_diffusion_defaults().keys()), conf=conf
    )
    model.load_state_dict(
        dist_util.load_state_dict(os.path.expanduser(
            conf.model_path), map_location="cpu")
    )
    model.to(device)
    if conf.use_fp16:
        model.convert_to_fp16()
    model.eval()

    show_progress = conf.show_progress

    if conf.classifier_scale > 0 and conf.classifier_path:
        print("loading classifier...")
        classifier = create_classifier(
            **select_args(conf, classifier_defaults().keys()))
        classifier.load_state_dict(
            dist_util.load_state_dict(os.path.expanduser(
                conf.classifier_path), map_location="cpu")
        )

        classifier.to(device)
        if conf.classifier_use_fp16:
            classifier.convert_to_fp16()
        classifier.eval()

        def cond_fn(x, t, y=None, gt=None, **kwargs):
            assert y is not None
            with th.enable_grad():
                x_in = x.detach().requires_grad_(True)
                logits = classifier(x_in, t)
                log_probs = F.log_softmax(logits, dim=-1)
                selected = log_probs[range(len(logits)), y.view(-1)]
                return th.autograd.grad(selected.sum(), x_in)[0] * conf.classifier_scale
    else:
        cond_fn = None

    def model_fn(x, t, y=None, gt=None, **kwargs):
        assert y is not None
        return model(x, t, y if conf.class_cond else None, gt=gt)

    print("sampling...")
    all_images = []

    dset = 'eval'

    eval_name = conf.get_default_eval_name()

    dl = conf.get_dataloader(dset=dset, dsName=eval_name)
    print(len(dl)) 
    for batch in iter(dl):

        for k in batch.keys():
            if isinstance(batch[k], th.Tensor):
                batch[k] = batch[k].to(device)

        model_kwargs = {}

        model_kwargs["gt"] = batch['GT']

        gt_keep_mask = batch.get('gt_keep_mask')
        if gt_keep_mask is not None:
            model_kwargs['gt_keep_mask'] = gt_keep_mask

        batch_size = model_kwargs["gt"].shape[0]

        if conf.cond_y is not None:
            classes = th.ones(batch_size, dtype=th.long, device=device)
            model_kwargs["y"] = classes * conf.cond_y
        else:
            classes = th.randint(
                low=0, high=NUM_CLASSES, size=(batch_size,), device=device
            )
            model_kwargs["y"] = classes

        sample_fn = (
            diffusion.p_sample_loop if not conf.use_ddim else diffusion.ddim_sample_loop
        )


        result = sample_fn(
            model_fn,
            (batch_size, 3, conf.image_size, conf.image_size),
            clip_denoised=conf.clip_denoised,
            model_kwargs=model_kwargs,
            cond_fn=cond_fn,
            device=device,
            progress=show_progress,
            return_all=True,
            conf=conf
        )
        srs = toU8(result['sample'])
        gts = toU8(result['gt'])
        lrs = toU8(result.get('gt') * model_kwargs.get('gt_keep_mask') + (-1) *
                   th.ones_like(result.get('gt')) * (1 - model_kwargs.get('gt_keep_mask')))

        gt_keep_masks = toU8((model_kwargs.get('gt_keep_mask') * 2 - 1))

        conf.eval_imswrite(
            srs=srs, gts=gts, lrs=lrs, gt_keep_masks=gt_keep_masks,
            img_names=batch['GT_name'], dset=dset, name=eval_name, verify_same=False)

    print("sampling complete, now check output in ./output folder")

print("First put that image in location ./Image_you_want_to_use ")
print("after that two window will open, you can paint mask on that selected image whatever part you want to remove and whenever you are done then simply press d")

First put that image in location ./Image_you_want_to_use 
after that two window will open, you can paint mask on that selected image whatever part you want to remove and whenever you are done then simply press d


In [2]:
def resize_image(image):
    return cv2.resize(image, (256, 256))

def paint(event, x, y, flags, param):
    global drawing

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        cv2.circle(selected_image, (x, y), brush_size, (0, 0, 0), -1)
        cv2.circle(white_image, (x, y), brush_size, (0, 0, 0), -1)

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            cv2.circle(selected_image, (x, y), brush_size, (0, 0, 0), -1)
            cv2.circle(white_image, (x, y), brush_size, (0, 0, 0), -1)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False

selected_image_filename = input("Enter the name of the image file which is in the location './Image you want to use' (including extension): ")
selected_image_path = os.path.join("./Image you want to use/", selected_image_filename)
selected_image = cv2.imread(selected_image_path)
if selected_image is None:
    print("Error: Image not found or could not be opened.")
    exit()

selected_image = resize_image(selected_image)
cv2.imwrite(selected_image_path, selected_image)

white_image = np.ones_like(selected_image) * 255

drawing = False
brush_size = 5

cv2.namedWindow('Painting')
cv2.namedWindow('Selected Image')

cv2.setMouseCallback('Selected Image', paint)

while True:
    cv2.imshow('Selected Image', selected_image)
    cv2.imshow('Painting', white_image)
    
    # Check for key press
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('d'):
        mask_filename = "mask.jpg"
        mask_path = os.path.join("./Image Mask You Want to use/", mask_filename)
        cv2.imwrite(mask_path, white_image)
        print("Mask saved as", mask_path)
        break

cv2.destroyAllWindows()

print("Make sure you have only 1-1 image in location './Image Mask you want to use ' and './Image you want to use' ")
print("If you want to use sample mask then you can simply take that mask image from ./data/dataset/() and paste in Image mask you want to use")

Enter the name of the image file which is in the location './Image you want to use' (including extension):  test.jpg


Mask saved as ./Image Mask You Want to use/mask.jpg
Make sure you have only 1-1 image in location './Image Mask you want to use ' and './Image you want to use' 
If you want to use sample mask then you can simply take that mask image from ./data/dataset/() and paste in Image mask you want to use


In [3]:
# Take an integer input from the user
user_input = int(input("if your image contain human then press 1 otherwise press 2 "))

# Check the user's input and set the conf_path accordingly
if user_input == 1:
    conf_path = "Real.yml"
elif user_input == 2:
    conf_path = "Real2.yml"
else:
    print("Invalid input. Please enter either 1 or 2.")
    exit()

print("Configuration path is:", conf_path)


if your image contain human then press 1 otherwise press 2  1


Configuration path is: Real.yml


In [5]:
#main(conf_path)